## GPT 2 Fine-tuned Model Story Generation

Last modified: 06-01-2024

Name: Mahamuge Dinendra Nadishan Costa

UWE Student Number: 13030224

In [ ]:
#Install the required library updates for both accelerate and transformers
#from huggingface
!pip install -U accelerate
!pip install -U transformers

# This resets the environment automatically to apply the installations above.
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Set the device to CUDA (or GPU) for processing
import torch

device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f'Using Device: {device}')

Using Device: cuda


In [3]:
#Import the required GPT2 modules from the transformers library for running
#GPT2 model
from transformers import GPT2Tokenizer,GPT2LMHeadModel,TrainingArguments,Trainer,DataCollatorWithPadding

from torch.utils.data import Dataset



# If a local copy of the fine-tuned model is unavailable, use the following two lines to load the model and tokenizer to load it directly from Huggingface:

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
# model = GPT2LMHeadModel.from_pretrained("gpt2-medium")



# Run the following to load the fine-tuned model on stories with original lengths
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Story Gen/GPT 2 Finetuned/tokenizer")
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Story Gen/GPT 2 Finetuned/model")


# Run following two lines to load the fine tuned model trained on equivalent length stories
# tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Story Gen/GPT 2 Finetuned with data balance/tokenizer")
# model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Story Gen/GPT 2 Finetuned with data balance/model")



if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.pad_token_id = tokenizer.eos_token_id

#Send the model to process within the GPU
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

## Evaluating the fine-tuned model with 10 prompts

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

#Set prompt text for model
prompt_text = "He drew his sword and pointed at the peculiar"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

He drew his sword and pointed at the peculiar creature, whose white, rounded face was hidden beneath two large bushy eyebrows and a mouth adorned by small teeth set in a line down the middle of its face—its jaw itself, a very unusual feature in these lower vertebrates, and which was adorned by two large, round teeth, one set in front and one set behind on both of its heads, and which were adorned with a yellow, fleshy band at the top and sides, a ring of bright, violet-hued feathers, a ring of silver on each side of its body, a fin exactly like that of the skiff of an eel or the argonauts, and various other appendages from the tail, wings, teeth, and so forth—all objects highly suggestive of some superhuman being, though none of them had the ring of a demon, and none were even faintly resembling the wings or the tail of a human being—and, finally, drew forth his crony of a brother, whom he met on the platform, not as the messenger of the sun or moon, but holding in one hand the weapon 

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

prompt_text = '''Nihara the warrior princess was calm in the face of danger.
No one dared challenge her now.
'''


#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)



print(generated_text)

Nihara the warrior princess was calm in the face of danger.
No one dared challenge her now.
“It will be easy for you, my lord! You will have the upper hand for some time, and nobody will want to draw swords against you when you have such formidable opponents!” It was not difficult to see what effect her courage had produced in this eccentric woman, who in the meantime, had begun to grow wise to a real art, and could not but perceive that she had won the hearts of her friends by a strange drama, when the old man’s words, full of gallantry, made her utter surprise and terror on seeing the effects of her audacity upon the young Musketeer, who, not only looked her fair in the face, but smiled on seeing the joy which was at his birth, and the gratitude she felt toward the great lord who inspired in her heart by having made her father, a handsome knight, and who was no doubt, at the time, one of the most illustrious men in Japan, the most exalted in his kingdom, and lord of a kingdom which, 

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

prompt_text = "And so it was the beginning of a beautiful friendship. Perhaps even something more. "

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

And so it was the beginning of a beautiful friendship. Perhaps even something more.     Pshaw! What can he be thinking, when we are so near a decision as to go with him to the ‘Nam?’? Not a thing, not a thought! You see now, my young friend, what a long, winding road it is to decide; and when you reach a decision, you can take a hundred opposite ways, and it is not probable you are ever going to reach a decision again; and you might as well adopt my plan as mine to-morrow, and tomorrow it would be so much wiser, because your friends will be as good or as bad as they are or are; so you have got to be patient!” “Do you believe so?” cried Andrew Stuart; and his eyes flashed round for a moment for a moment into icy fire, and then fell without another instant in the depths of his heart, as one resigned himself to wait till they were sure they were no more likely than not to be at all sorry for having to deal with a person who, for an instant, he felt as good as if his life depended upon him

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

prompt_text = "Thine eyes shine like bright diamonds"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Thine eyes shine like bright diamonds; Thy hair is golden; thy tongue doth quench the blood which doth quench thine open wounds.—Thou art the Prince of Verona, which hath slain Romeo, and Romeo died.—The Prince of Verona shall live; but thou shalt not have power to slay the King of Scotland, Who was thy father, and hadst thou been but half of his height, Thou shouldst have killed him, and taken his crown, I should know it’d I should kill thee.— Romeo,” said she, in anger, “remember how it is that in thy life thou play’st, and when thou hast done, It is like a wretch that lives but a month longer.—Why, thou do’t forget.—Thou wouldst slay me.— I will not forget.— And if thou hadst but one wish, That I could bestow on thee, It would be my death, for it were the happiest of all That thou mayst have.—If thou hadst but one thing, and that was death, thou wouldst deny It to any but my cousin.— But now I’ll speak one thing, and I would tell thee that love, Hath but one name, and that is sister

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

prompt_text = "Fly you fools!"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Fly you fools! I will have you arrested, I do entreat you! “ “ I pardon you,” cried Buckingham; “ pardon you, then, my dear love, pardon you, and I do love you!” “Be satisfied; you will see to it that everything is at the least possible; I am not at all guilty of your anger, although I know what is in your heart.” “How, you know what?” “Oh, I—I know how your love comes from you; I know all that is in the heart of a lion; I know what your grief is—even what you would not dare to tell me! You know—” “Yes; but you know also that I love you, since you never forget what I have told you—even when I send you my last words, I love you, since I love you, and yet you forget! You know, my dear Constance! Let it suffice, then; you shall love me as I love you.” “I will not lose my time in thinking how you would feel when my executioner came for you, you know; at least I shall not forget you—for as soon as he appears in the world you will remember me.” “But you, what would you say to an executioner 

In [ ]:
#Set the model to evaluate to run prompts on the model for generation
model.eval()

prompt_text = "The underground passage led to heaven"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 2.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

The underground passage led to heaven and back again, to the world of the living and the dead, and to the world of the angels, to the world of the devils, to the world of the animals, to the world of the ants and the flies, to the world of the tiny creatures, to the world of the ants and the flies, and so forth; to the whole infinite vastness of space, and back again and again to the world of the living and the dead, to the earth as a whole, to the sun and the moon, to the whole vastness of time, and back to the world of the ants and the flies! It was the most wonderful paradox, the most wonderful paradox in the whole Book of Ecclesiastes! It was a paradox, a paradox so profound and profound that for a time the mind of the young man was darkened by it, and he sank deeper and deeper into reverie, while the beautiful woman sat in the armchair by the fire and listened to this profound reverie with her eyes shut and her hands stretched out upon the table; and at length he began to think th

In [ ]:
model.eval()

prompt_text = "The ray gun vaporized the entire town under the sea"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

The ray gun vaporized the entire town under the sea’s intense rays, leaving little in its way but smoky black ruins that I couldn’t linger in; the smoke obscured the moon’s light, hid the horizon with an atmospheric haze, and made our way through a carpet of crimson and crimson gas that drifted over the waves to form a brilliant halo over the waters, lighting up my way like an illuminator hung with threads of incandescent light! Before long, I could see every street, every low street corner, and every house on every street and in every castle on every hill, even the little ones with their low entryways, their few verandas, their modest lawns gleaming in the sunlight! For a while I stood amazed at that dazzling effect, marveling at its power, when, half-fascinated, I saw only a dark shadowed horizon stretching between the towers of one of the town’s tallest buildings, its black spire, with its minarets and gables; a mere fringe of sky above its parapet of red roofs that masked the full 

In [4]:
model.eval()

prompt_text = "Kushan, Nadishan and Dilshani were primordial entities that defended the very fabric of reality"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Kushan, Nadishan and Dilshani were primordial entities that defended the very fabric of reality with an indomitable vigor unrivaled by any of the primordial races on this earth—evidently, the giants, the monkeys, the sheep, the ants, the flies, and reptiles of the earth! What an extraordinary vigor in defending itself against the invaders of space that, at the end of one month, the whole earth had yielded! During the last three months of March, no less than ten large ships had encountered this prodigious narwhale: the China, the Russia, the Helvetia, the CASSANDRA, the Mauritius, the Mauritius IIII, the Java, and finally, that mysterious frigate, the Abraham Lincoln! Ten vessels? Impossible! The Abraham Lincoln, commander of, I fear, the most prodigious whaling vessel afloat in the world, had only to deal with one whale every three months! But this prodigious narwhale had increased in speed since it left the seas of Japan almost a year ago! During that same period of time, the seas of 

In [ ]:
model.eval()

prompt_text = '''Nihara, Suj and Janu wielded the powers of the kingdom. They were the fairest and most beautiful princesses ever seen. '''

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 3.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Nihara, Suj and Janu wielded the powers of the kingdom. They were the fairest and most beautiful princesses ever seen.  [Author’s Note: “The Japanese use the word “karawada, or “karanawada.” It can also be read as “girlage,” and as a translation of the Greek word, which means “crowned,” or “noble princess.”]” “Then,” said the judge, “if the three of them will come to the castle today, I wish them my daughter, my precious child, and they shall inherit my kingdom and all its treasures, as you wish.” The three of them bowed prodigiously, and the judge, without giving his reasons, dismissed them all with an air of serenity—a sentiment which reassured and comforted the three girls more than the sentence they had proposed to give in order to win their freedom—and went away to lead the search of the princesses after their new-found benefactress and her new-found benefactress’s child! On the fifth day the three young fellows were gone—sent by the kingdom to pursue the three enchantresses; but 

In [ ]:
model.eval()

prompt_text = 'Aragon and Frodo along with Passepartout leapt from the plane'

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 6.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Aragon and Frodo along with Passepartout leapt from the plane and came down a short, light, dusty street, before which stood a pavilion bearing an arborized arched entry like those used at Paris or Amiens, but with an inscription ARCHEMITH, as on a church rather than an office stand; for at the crossroads ran a large door of a curious species of arbor mold that was evidently at first a door in wooden construction; the mold of the windows around it glittered with diamonds, though the light was a pale blue color and disappeared in a pale twilight into the blackness of night, with a brightness less intense than that of daylight within a chamber suspended above a depth of dim darkness; and to the right an immense door of an unknown mold, though open to anybody who sought it with the least task, let them pass before the pavilion they sought, turning a corner of this immense gateway into the light by means of an immense lantern from which their passage was illuminated from two lower stories—

In [ ]:
model.eval()

prompt_text = "Great Scott!"

#Convert the prompt text to tokens and add the attention mask
input_ids = tokenizer(prompt_text, return_tensors = "pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt"
).attention_mask

#Send converted prompt and attention mask to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


#Generate the output from the model based on the parameter set here
output = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length = 1024,
    num_beams = 10,
    min_length = 1000,
    temperature = 6.5,
    top_k = 50,
    do_sample = True,
)


generated_text = tokenizer.decode(output[0],skip_special_tokens = True)

print(generated_text)

Great Scott! If you can but walk, As soon shall we send you aboard With Gretel; she shall be better Nurse Than any else I have seen: and as you have a new tooth, She says your name first, and says that you have met with a villainous purpose: I give you the goose, and when you find She wants her, you can kill the grinder.’ “What villain has done this?” thought the queen, still in a great wonder; her heart beat So very heavy that her mouth quivered, For the sight of her grief might destroy her features like lightning: So, if she will but eat some root, Her hunger must cease: her mouth would seem like an empty jar, And her brain seem like a dead man; who will dare To think or act: what a misfortune it is By letting her live and thinking with her; what good can a king’s love bring In thinking or a love deed Have? She ate her spinach with her toast: but, when her fowl Lid fell upon golden brown, Her heart was too painful to draw forth any drop of water, Till on her golden cup there quivered